### 카메라로 박스의 크기를 추정한게 정확한가?

In [1]:
import numpy as np
import pyrealsense2 as rs
from ultralytics import YOLO
import time
from collections import deque

MODEL_PATH = "/home/dw/ws_job_msislab/amr_project/src/runs/obb/smoke_test_v2/weights/best.pt"

CONF_THRES = 0.85
IOU_THRES  = 0.75
IMGSZ      = 640

DEPTH_WIN = 7
AVG_N = 10
TIMEOUT_SEC = 25.0  # 유효값 10개 모을 때까지 최대 대기

# -----------------------------
# ✅ Size Outlier Filter (cm)
#   너 로그 기준(대부분 23x12 근처)에서 튀는 애(58x46 등) 제거용
#   필요하면 범위만 조절하면 됨.
# -----------------------------
LONG_RANGE_CM  = (10.0, 40.0)   # long 변 허용 범위
SHORT_RANGE_CM = (5.0, 25.0)    # short 변 허용 범위
RATIO_RANGE    = (1.2, 4.5)     # long/short 비율 허용 범위

def valid_size(long_cm, short_cm,
               long_range=LONG_RANGE_CM,
               short_range=SHORT_RANGE_CM,
               ratio_range=RATIO_RANGE):
    if long_cm <= 0 or short_cm <= 0:
        return False
    if not (long_range[0] <= long_cm <= long_range[1]):
        return False
    if not (short_range[0] <= short_cm <= short_range[1]):
        return False
    r = long_cm / max(1e-6, short_cm)
    if not (ratio_range[0] <= r <= ratio_range[1]):
        return False
    return True

def clamp(v, lo, hi):
    return max(lo, min(hi, v))

def get_depth_median_around(depth_frame, cx, cy, win=7):
    half = win // 2
    ds = []
    for dy in range(-half, half + 1):
        for dx in range(-half, half + 1):
            x = cx + dx
            y = cy + dy
            try:
                d = depth_frame.get_distance(x, y)
            except:
                d = 0.0
            if d > 0.0:
                ds.append(d)
    if not ds:
        return 0.0
    return float(np.median(ds))

def obb_angle_deg_upright0_rightplus(poly4x2: np.ndarray) -> float:
    """
    upright=0, right +, left -
    (네가 쓰던 PCA 방식 그대로)
    """
    p = poly4x2.astype(np.float32)
    c = p.mean(axis=0, keepdims=True)
    q = p - c

    cov = np.cov(q.T)
    eigvals, eigvecs = np.linalg.eig(cov)
    v = eigvecs[:, np.argmax(eigvals)].astype(np.float32)

    vx, vy = float(v[0]), float(v[1])
    if vy < 0:
        vx, vy = -vx, -vy

    angle = float(np.degrees(np.arctan2(vx, vy)))
    angle = -angle
    return angle

def estimate_obb_size_cm(depth_frame, intr, poly4x2, w, h, win=7):
    """
    OBB 4점 각각 deproject → 3D 변 길이(cm)
    return: (long_cm, short_cm, ok, edges_cm[4])
    """
    pts2d = poly4x2.astype(np.float32)
    pts3d = []

    for (x, y) in pts2d:
        cx = clamp(int(round(x)), 0, w - 1)
        cy = clamp(int(round(y)), 0, h - 1)

        d = depth_frame.get_distance(cx, cy)
        if d <= 0.0:
            d = get_depth_median_around(depth_frame, cx, cy, win=win)
        if d <= 0.0:
            return 0.0, 0.0, False, None

        X, Y, Z = rs.rs2_deproject_pixel_to_point(intr, [cx, cy], d)
        pts3d.append(np.array([X, Y, Z], dtype=np.float32))

    p0, p1, p2, p3 = pts3d

    def dist_cm(a, b):
        return float(np.linalg.norm(a - b)) * 100.0

    d01 = dist_cm(p0, p1)
    d12 = dist_cm(p1, p2)
    d23 = dist_cm(p2, p3)
    d30 = dist_cm(p3, p0)

    # 마주보는 변 평균으로 안정화
    a = 0.5 * (d01 + d23)
    b = 0.5 * (d12 + d30)

    long_cm = max(a, b)
    short_cm = min(a, b)
    return long_cm, short_cm, True, [d01, d12, d23, d30]

def main():
    model = YOLO(MODEL_PATH)
    print("[INFO] Model loaded:", MODEL_PATH)
    print(f"[INFO] conf>={CONF_THRES}, iou={IOU_THRES}, imgsz={IMGSZ}")
    print(f"[INFO] Need {AVG_N} valid frames. Timeout={TIMEOUT_SEC}s")
    print(f"[INFO] size filter: long{LONG_RANGE_CM} short{SHORT_RANGE_CM} ratio{RATIO_RANGE}\n")

    pipeline = rs.pipeline()
    config = rs.config()

    width, height, fps = 640, 480, 30
    config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, fps)
    config.enable_stream(rs.stream.depth, width, height, rs.format.z16, fps)

    pipeline.start(config)
    align = rs.align(rs.stream.color)

    # buf: (Xcm, Ycm, Zcm, distcm, angledeg, longcm, shortcm)
    buf = deque(maxlen=AVG_N)
    t0 = time.time()

    try:
        while True:
            if time.time() - t0 > TIMEOUT_SEC:
                print("[FAIL] Timeout: not enough valid frames.")
                break

            frames = pipeline.wait_for_frames()
            frames = align.process(frames)

            color_frame = frames.get_color_frame()
            depth_frame = frames.get_depth_frame()
            if not color_frame or not depth_frame:
                continue

            frame = np.asanyarray(color_frame.get_data())
            intr = color_frame.profile.as_video_stream_profile().get_intrinsics()

            results = model.predict(
                frame,
                imgsz=IMGSZ,
                conf=CONF_THRES,
                iou=IOU_THRES,
                verbose=False
            )
            r = results[0]

            # best 1개 선택
            best = None  # (conf, cls, poly4x2)
            if getattr(r, "obb", None) is not None and r.obb is not None:
                obb = r.obb
                if obb.xyxyxyxy is not None and len(obb.xyxyxyxy) > 0:
                    polys = obb.xyxyxyxy.cpu().numpy()
                    confs = obb.conf.cpu().numpy().astype(float)
                    clss  = obb.cls.cpu().numpy().astype(int)

                    keep = confs >= CONF_THRES
                    for poly8, cf, ci in zip(polys[keep], confs[keep], clss[keep]):
                        if best is None or cf > best[0]:
                            best = (cf, ci, poly8.reshape(4, 2))

            if best is None:
                continue

            cf, ci, poly = best

            # center px
            cx = int(np.mean(poly[:, 0]))
            cy = int(np.mean(poly[:, 1]))
            cx = clamp(cx, 0, width - 1)
            cy = clamp(cy, 0, height - 1)

            # center depth
            depth_m = depth_frame.get_distance(cx, cy)
            if depth_m <= 0.0:
                depth_m = get_depth_median_around(depth_frame, cx, cy, win=DEPTH_WIN)
            if depth_m <= 0.0:
                continue

            # center 3D
            X, Y, Z = rs.rs2_deproject_pixel_to_point(intr, [cx, cy], depth_m)
            dist = float(np.sqrt(X*X + Y*Y + Z*Z))
            angle = obb_angle_deg_upright0_rightplus(poly)

            # size from corners
            long_cm, short_cm, ok_size, edges = estimate_obb_size_cm(
                depth_frame, intr, poly, width, height, win=DEPTH_WIN
            )
            if not ok_size:
                continue

            # ✅ outlier filter
            if not valid_size(long_cm, short_cm):
                print(f"[SKIP] size outlier: {long_cm:.1f}x{short_cm:.1f} cm  (conf={cf:.2f})")
                continue

            Xcm, Ycm, Zcm = X*100.0, Y*100.0, Z*100.0
            distcm = dist*100.0

            buf.append((Xcm, Ycm, Zcm, distcm, angle, long_cm, short_cm))

            print(f"[{len(buf)}/{AVG_N}] conf={cf:.2f}  XYZ(cm)=({Xcm:+.2f},{Ycm:+.2f},{Zcm:+.2f})  "
                  f"angle={angle:+.2f}  size≈{long_cm:.1f}x{short_cm:.1f}cm")

            if len(buf) == AVG_N:
                arr = np.array(buf, dtype=np.float32)
                mx, my, mz, md, ma, mlong, mshort = arr.mean(axis=0)

                print("\n========== RESULT (AVERAGE over 10 valid) ==========")
                print(f"count            : {AVG_N}")
                print(f"cam XYZ avg (cm) : ({mx:+.2f}, {my:+.2f}, {mz:+.2f})")
                print(f"dist_avg (cm)    : {md:.2f}")
                print(f"angle_avg (deg)  : {ma:+.2f}  (upright=0, right +, left -)")
                print(f"size_avg (cm)    : {mlong:.1f} x {mshort:.1f}  (long x short)")
                print("====================================================\n")
                break

    finally:
        pipeline.stop()

if __name__ == "__main__":
    main()


[INFO] Model loaded: /home/dw/ws_job_msislab/amr_project/src/runs/obb/smoke_test_v2/weights/best.pt
[INFO] conf>=0.85, iou=0.75, imgsz=640
[INFO] Need 10 valid frames. Timeout=25.0s
[INFO] size filter: long(10.0, 40.0) short(5.0, 25.0) ratio(1.2, 4.5)

[1/10] conf=0.93  XYZ(cm)=(+3.15,-9.73,+36.50)  angle=+1.30  size≈23.0x10.2cm
[2/10] conf=0.92  XYZ(cm)=(+3.24,-9.72,+36.44)  angle=+1.27  size≈23.1x10.2cm
[3/10] conf=0.92  XYZ(cm)=(+3.34,-10.03,+37.61)  angle=+1.21  size≈23.0x10.2cm
[SKIP] size outlier: 72.5x59.9 cm  (conf=0.92)
[4/10] conf=0.92  XYZ(cm)=(+3.29,-9.87,+36.99)  angle=+1.27  size≈23.1x10.2cm
[SKIP] size outlier: 42.5x29.6 cm  (conf=0.92)
[SKIP] size outlier: 66.5x54.3 cm  (conf=0.92)
[5/10] conf=0.92  XYZ(cm)=(+3.16,-9.49,+35.57)  angle=+1.17  size≈23.0x10.2cm
[6/10] conf=0.93  XYZ(cm)=(+3.15,-9.46,+35.46)  angle=+1.43  size≈23.0x10.2cm
[7/10] conf=0.92  XYZ(cm)=(+3.27,-9.81,+36.80)  angle=+1.22  size≈23.1x10.2cm
[8/10] conf=0.92  XYZ(cm)=(+3.17,-9.50,+35.63)  angle=+1.20

### 2. 화면에 띄우기

In [2]:
import numpy as np
import pyrealsense2 as rs
from ultralytics import YOLO
import cv2
import time

MODEL_PATH = "/home/dw/ws_job_msislab/amr_project/src/runs/obb/smoke_test_v2/weights/best.pt"

CONF_THRES = 0.85
IOU_THRES  = 0.75
IMGSZ      = 640

DEPTH_WIN = 7  # center depth가 0이면 주변 median으로 보정

def clamp(v, lo, hi):
    return max(lo, min(hi, v))

def get_depth_median_around(depth_frame, cx, cy, win=7):
    half = win // 2
    ds = []
    for dy in range(-half, half + 1):
        for dx in range(-half, half + 1):
            x = cx + dx
            y = cy + dy
            try:
                d = depth_frame.get_distance(x, y)
            except:
                d = 0.0
            if d > 0.0:
                ds.append(d)
    return float(np.median(ds)) if ds else 0.0

def obb_angle_deg_upright0_rightplus(poly4x2: np.ndarray) -> float:
    p = poly4x2.astype(np.float32)
    c = p.mean(axis=0, keepdims=True)
    q = p - c

    cov = np.cov(q.T)
    eigvals, eigvecs = np.linalg.eig(cov)
    v = eigvecs[:, np.argmax(eigvals)].astype(np.float32)

    vx, vy = float(v[0]), float(v[1])
    if vy < 0:
        vx, vy = -vx, -vy

    angle = float(np.degrees(np.arctan2(vx, vy)))
    angle = -angle
    return angle

def draw_hud(img, lines, x=10, y=10, line_h=24):
    # 반투명 배경 박스
    pad = 8
    w = max([cv2.getTextSize(s, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0][0] for s in lines] + [10])
    h = line_h * len(lines)
    x2, y2 = x + w + pad*2, y + h + pad*2

    overlay = img.copy()
    cv2.rectangle(overlay, (x, y), (x2, y2), (0, 0, 0), -1)
    cv2.addWeighted(overlay, 0.45, img, 0.55, 0, img)

    # 텍스트
    ty = y + pad + 18
    for s in lines:
        cv2.putText(img, s, (x + pad, ty), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
        ty += line_h

def main():
    model = YOLO(MODEL_PATH)
    print("[INFO] Model loaded:", MODEL_PATH)
    print(f"[INFO] conf>={CONF_THRES}, iou={IOU_THRES}, imgsz={IMGSZ}")
    print("[INFO] ESC to quit")

    pipeline = rs.pipeline()
    config = rs.config()

    width, height, fps = 640, 480, 30
    config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, fps)
    config.enable_stream(rs.stream.depth, width, height, rs.format.z16, fps)

    pipeline.start(config)
    align = rs.align(rs.stream.color)

    cv2.namedWindow("OBB Live", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("OBB Live", width, height)

    # ✅ 마지막 유효 값(항상 표시용)
    last = {
        "ok": False,
        "Xcm": 0.0, "Ycm": 0.0, "Zcm": 0.0,
        "distcm": 0.0,
        "angle": 0.0,
        "conf": 0.0,
        "cls": -1,
        "t": 0.0,          # last update time
        "cx": width // 2,
        "cy": height // 2,
    }

    try:
        while True:
            frames = pipeline.wait_for_frames()
            frames = align.process(frames)

            color_frame = frames.get_color_frame()
            depth_frame = frames.get_depth_frame()
            if not color_frame or not depth_frame:
                continue

            frame = np.asanyarray(color_frame.get_data())
            intr = color_frame.profile.as_video_stream_profile().get_intrinsics()

            vis = frame.copy()
            status = "WAIT"

            # YOLO OBB inference
            results = model.predict(
                frame,
                imgsz=IMGSZ,
                conf=CONF_THRES,
                iou=IOU_THRES,
                verbose=False
            )
            r = results[0]

            best = None  # (conf, cls, poly4x2)
            if getattr(r, "obb", None) is not None and r.obb is not None:
                obb = r.obb
                if obb.xyxyxyxy is not None and len(obb.xyxyxyxy) > 0:
                    polys = obb.xyxyxyxy.cpu().numpy()
                    confs = obb.conf.cpu().numpy().astype(float)
                    clss  = obb.cls.cpu().numpy().astype(int)

                    # conf 필터는 여기서만
                    keep = confs >= CONF_THRES
                    for poly8, cf, ci in zip(polys[keep], confs[keep], clss[keep]):
                        if best is None or cf > best[0]:
                            best = (cf, ci, poly8.reshape(4, 2))

            if best is not None:
                cf, ci, poly = best

                # OBB 그리기
                poly_i = np.round(poly).astype(np.int32).reshape(-1, 1, 2)
                cv2.polylines(vis, [poly_i], True, (0, 255, 0), 2)

                cx = int(np.mean(poly[:, 0]))
                cy = int(np.mean(poly[:, 1]))
                cx = clamp(cx, 0, width - 1)
                cy = clamp(cy, 0, height - 1)
                cv2.circle(vis, (cx, cy), 4, (0, 0, 255), -1)

                # depth at center
                depth_m = depth_frame.get_distance(cx, cy)
                if depth_m <= 0.0:
                    depth_m = get_depth_median_around(depth_frame, cx, cy, win=DEPTH_WIN)

                if depth_m > 0.0:
                    X, Y, Z = rs.rs2_deproject_pixel_to_point(intr, [cx, cy], depth_m)
                    dist = float(np.sqrt(X*X + Y*Y + Z*Z))
                    angle = obb_angle_deg_upright0_rightplus(poly)

                    # last 업데이트
                    last["ok"] = True
                    last["Xcm"], last["Ycm"], last["Zcm"] = X*100.0, Y*100.0, Z*100.0
                    last["distcm"] = dist*100.0
                    last["angle"] = angle
                    last["conf"] = float(cf)
                    last["cls"] = int(ci)
                    last["t"] = time.time()
                    last["cx"], last["cy"] = cx, cy

                    status = "OK"
                else:
                    status = "DEPTH_INVALID"
            else:
                status = "NO_DET"

            # ✅ 항상 HUD 표시 (탐지 실패해도 last 값 유지)
            age = time.time() - last["t"] if last["ok"] else 999.0
            stale = "STALE" if (not last["ok"] or age > 0.5) else "LIVE"

            hud_lines = [
                f"status: {status} / {stale} (age={age:.2f}s)",
                f"conf={last['conf']:.2f}  cls={last['cls']}",
                f"cam XYZ(cm)=({last['Xcm']:+.2f}, {last['Ycm']:+.2f}, {last['Zcm']:+.2f})",
                f"dist={last['distcm']:.2f} cm   angle={last['angle']:+.2f} deg",
                f"center px=({last['cx']},{last['cy']})",
            ]
            draw_hud(vis, hud_lines)

            cv2.imshow("OBB Live", vis)

            key = cv2.waitKey(1) & 0xFF
            if key == 27:  # ESC
                break

    finally:
        pipeline.stop()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


[INFO] Model loaded: /home/dw/ws_job_msislab/amr_project/src/runs/obb/smoke_test_v2/weights/best.pt
[INFO] conf>=0.85, iou=0.75, imgsz=640
[INFO] ESC to quit


### 3.10번 평균값 반환하기

In [4]:
import numpy as np
import pyrealsense2 as rs
from ultralytics import YOLO
import time
from collections import deque

MODEL_PATH = "/home/dw/ws_job_msislab/amr_project/src/runs/obb/smoke_test_v2/weights/best.pt"

CONF_THRES = 0.85
IOU_THRES  = 0.75
IMGSZ      = 640

DEPTH_WIN = 7
AVG_N = 10
TIMEOUT_SEC = 25.0

# -----------------------------
# ✅ Size Outlier Filter (cm)
# -----------------------------
LONG_RANGE_CM  = (10.0, 40.0)
SHORT_RANGE_CM = (5.0, 25.0)
RATIO_RANGE    = (1.2, 4.5)

# -----------------------------
# ✅ Robot Offset (cm)
#   그리퍼 = 카메라 기준
#   y: -7cm, z: -18cm
# -----------------------------
OFF_X_CM = 0.0
OFF_Y_CM = -7.0
OFF_Z_CM = -18.0

def valid_size(long_cm, short_cm,
               long_range=LONG_RANGE_CM,
               short_range=SHORT_RANGE_CM,
               ratio_range=RATIO_RANGE):
    if long_cm <= 0 or short_cm <= 0:
        return False
    if not (long_range[0] <= long_cm <= long_range[1]):
        return False
    if not (short_range[0] <= short_cm <= short_range[1]):
        return False
    r = long_cm / max(1e-6, short_cm)
    if not (ratio_range[0] <= r <= ratio_range[1]):
        return False
    return True

def clamp(v, lo, hi):
    return max(lo, min(hi, v))

def get_depth_median_around(depth_frame, cx, cy, win=7):
    half = win // 2
    ds = []
    for dy in range(-half, half + 1):
        for dx in range(-half, half + 1):
            x = cx + dx
            y = cy + dy
            try:
                d = depth_frame.get_distance(x, y)
            except:
                d = 0.0
            if d > 0.0:
                ds.append(d)
    if not ds:
        return 0.0
    return float(np.median(ds))

def obb_angle_deg_upright0_rightplus(poly4x2: np.ndarray) -> float:
    p = poly4x2.astype(np.float32)
    c = p.mean(axis=0, keepdims=True)
    q = p - c

    cov = np.cov(q.T)
    eigvals, eigvecs = np.linalg.eig(cov)
    v = eigvecs[:, np.argmax(eigvals)].astype(np.float32)

    vx, vy = float(v[0]), float(v[1])
    if vy < 0:
        vx, vy = -vx, -vy

    angle = float(np.degrees(np.arctan2(vx, vy)))
    angle = -angle
    return angle

def estimate_obb_size_cm(depth_frame, intr, poly4x2, w, h, win=7):
    pts2d = poly4x2.astype(np.float32)
    pts3d = []

    for (x, y) in pts2d:
        cx = clamp(int(round(x)), 0, w - 1)
        cy = clamp(int(round(y)), 0, h - 1)

        d = depth_frame.get_distance(cx, cy)
        if d <= 0.0:
            d = get_depth_median_around(depth_frame, cx, cy, win=win)
        if d <= 0.0:
            return 0.0, 0.0, False, None

        X, Y, Z = rs.rs2_deproject_pixel_to_point(intr, [cx, cy], d)
        pts3d.append(np.array([X, Y, Z], dtype=np.float32))

    p0, p1, p2, p3 = pts3d

    def dist_cm(a, b):
        return float(np.linalg.norm(a - b)) * 100.0

    d01 = dist_cm(p0, p1)
    d12 = dist_cm(p1, p2)
    d23 = dist_cm(p2, p3)
    d30 = dist_cm(p3, p0)

    a = 0.5 * (d01 + d23)
    b = 0.5 * (d12 + d30)

    long_cm = max(a, b)
    short_cm = min(a, b)
    return long_cm, short_cm, True, [d01, d12, d23, d30]

def main():
    model = YOLO(MODEL_PATH)
    print("[INFO] Model loaded:", MODEL_PATH)
    print(f"[INFO] conf>={CONF_THRES}, iou={IOU_THRES}, imgsz={IMGSZ}")
    print(f"[INFO] Need {AVG_N} valid frames. Timeout={TIMEOUT_SEC}s")
    print(f"[INFO] size filter: long{LONG_RANGE_CM} short{SHORT_RANGE_CM} ratio{RATIO_RANGE}")
    print(f"[INFO] robot offset (cm): X{OFF_X_CM:+.1f}  Y{OFF_Y_CM:+.1f}  Z{OFF_Z_CM:+.1f}\n")

    pipeline = rs.pipeline()
    config = rs.config()

    width, height, fps = 640, 480, 30
    config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, fps)
    config.enable_stream(rs.stream.depth, width, height, rs.format.z16, fps)

    pipeline.start(config)
    align = rs.align(rs.stream.color)

    buf = deque(maxlen=AVG_N)  # (Xcm, Ycm, Zcm, distcm, angledeg, longcm, shortcm)
    t0 = time.time()

    try:
        while True:
            if time.time() - t0 > TIMEOUT_SEC:
                print("[FAIL] Timeout: not enough valid frames.")
                break

            frames = pipeline.wait_for_frames()
            frames = align.process(frames)

            color_frame = frames.get_color_frame()
            depth_frame = frames.get_depth_frame()
            if not color_frame or not depth_frame:
                continue

            frame = np.asanyarray(color_frame.get_data())
            intr = color_frame.profile.as_video_stream_profile().get_intrinsics()

            results = model.predict(
                frame,
                imgsz=IMGSZ,
                conf=CONF_THRES,
                iou=IOU_THRES,
                verbose=False
            )
            r = results[0]

            best = None  # (conf, cls, poly4x2)
            if getattr(r, "obb", None) is not None and r.obb is not None:
                obb = r.obb
                if obb.xyxyxyxy is not None and len(obb.xyxyxyxy) > 0:
                    polys = obb.xyxyxyxy.cpu().numpy()
                    confs = obb.conf.cpu().numpy().astype(float)
                    clss  = obb.cls.cpu().numpy().astype(int)

                    keep = confs >= CONF_THRES
                    for poly8, cf, ci in zip(polys[keep], confs[keep], clss[keep]):
                        if best is None or cf > best[0]:
                            best = (cf, ci, poly8.reshape(4, 2))

            if best is None:
                continue

            cf, ci, poly = best

            cx = int(np.mean(poly[:, 0]))
            cy = int(np.mean(poly[:, 1]))
            cx = clamp(cx, 0, width - 1)
            cy = clamp(cy, 0, height - 1)

            depth_m = depth_frame.get_distance(cx, cy)
            if depth_m <= 0.0:
                depth_m = get_depth_median_around(depth_frame, cx, cy, win=DEPTH_WIN)
            if depth_m <= 0.0:
                continue

            X, Y, Z = rs.rs2_deproject_pixel_to_point(intr, [cx, cy], depth_m)
            dist = float(np.sqrt(X*X + Y*Y + Z*Z))
            angle = obb_angle_deg_upright0_rightplus(poly)

            long_cm, short_cm, ok_size, edges = estimate_obb_size_cm(
                depth_frame, intr, poly, width, height, win=DEPTH_WIN
            )
            if not ok_size:
                continue

            if not valid_size(long_cm, short_cm):
                print(f"[SKIP] size outlier: {long_cm:.1f}x{short_cm:.1f} cm  (conf={cf:.2f})")
                continue

            Xcm, Ycm, Zcm = X*100.0, Y*100.0, Z*100.0
            distcm = dist*100.0

            buf.append((Xcm, Ycm, Zcm, distcm, angle, long_cm, short_cm))

            print(f"[{len(buf)}/{AVG_N}] conf={cf:.2f}  XYZ(cm)=({Xcm:+.2f},{Ycm:+.2f},{Zcm:+.2f})  "
                  f"angle={angle:+.2f}  size≈{long_cm:.1f}x{short_cm:.1f}cm")

            if len(buf) == AVG_N:
                arr = np.array(buf, dtype=np.float32)
                mx, my, mz, md, ma, mlong, mshort = arr.mean(axis=0)

                # ✅ 로봇 오프셋 적용 (cm)
                rx = mx + OFF_X_CM
                ry = my + OFF_Y_CM
                rz = mz + OFF_Z_CM

                print("\n========== RESULT (AVERAGE over 10 valid) ==========")
                print(f"count            : {AVG_N}")
                print(f"cam XYZ avg (cm) : ({mx:+.2f}, {my:+.2f}, {mz:+.2f})")
                print(f"dist_avg (cm)    : {md:.2f}")
                print(f"angle_avg (deg)  : {ma:+.2f}  (upright=0, right +, left -)")
                print(f"size_avg (cm)    : {mlong:.1f} x {mshort:.1f}  (long x short)")
                print("====================================================")

                # ✅ 요청: 한 줄 더 출력
                print(f"for robot offset : XYZ(cm)=({rx:+.2f}, {ry:+.2f}, {rz:+.2f})  "
                      f"(applied Y{OFF_Y_CM:+.1f}, Z{OFF_Z_CM:+.1f})\n")
                break

    finally:
        pipeline.stop()

if __name__ == "__main__":
    main()


[INFO] Model loaded: /home/dw/ws_job_msislab/amr_project/src/runs/obb/smoke_test_v2/weights/best.pt
[INFO] conf>=0.85, iou=0.75, imgsz=640
[INFO] Need 10 valid frames. Timeout=25.0s
[INFO] size filter: long(10.0, 40.0) short(5.0, 25.0) ratio(1.2, 4.5)
[INFO] robot offset (cm): X+0.0  Y-7.0  Z-18.0

[1/10] conf=0.96  XYZ(cm)=(+6.15,-10.43,+35.97)  angle=+33.88  size≈23.7x9.4cm
[2/10] conf=0.96  XYZ(cm)=(+5.95,-10.53,+35.35)  angle=+33.55  size≈23.7x9.5cm
[3/10] conf=0.95  XYZ(cm)=(+5.99,-10.77,+36.15)  angle=+33.12  size≈24.1x9.8cm
[SKIP] size outlier: 72.5x60.0 cm  (conf=0.95)
[4/10] conf=0.95  XYZ(cm)=(+5.78,-10.02,+34.85)  angle=+34.16  size≈23.0x9.6cm
[SKIP] size outlier: 73.3x60.4 cm  (conf=0.95)
[SKIP] size outlier: 77.5x64.4 cm  (conf=0.95)
[5/10] conf=0.95  XYZ(cm)=(+5.91,-10.15,+35.63)  angle=+34.44  size≈23.1x9.6cm
[6/10] conf=0.95  XYZ(cm)=(+5.72,-10.06,+35.63)  angle=+34.54  size≈23.0x9.9cm
[7/10] conf=0.95  XYZ(cm)=(+5.92,-10.02,+35.18)  angle=+34.50  size≈23.0x9.5cm
[SKIP]